## Importing necessary libraries

In [0]:
!pip install bs4

In [1]:
#import generic libraries
from google.colab import drive,files
import zipfile
import io
import os
import random
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

#String pre-processing
import re, string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
import csv
from tqdm import tqdm

# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
import csv
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Mounting drive and uploading data to google drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Setting Project Path

In [0]:
project_path='/content/drive/My Drive/SNLP_R9_Project'

In [0]:
os.makedirs(project_path)

In [0]:
os.chdir(project_path)

Upload zip file and extract

In [0]:
uploaded = files.upload()
data = zipfile.ZipFile(io.BytesIO(uploaded['blog-authorship-corpus.zip']), 'r')
data.extractall(project_path)

## Loading CSV file

In [0]:
blog_df=pd.read_csv(project_path+'/blogtext.csv')

In [6]:
blog_df.shape

(681284, 7)

In [0]:
blog_df.topic.value_counts()

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [0]:
blog_df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


# Preprocess rows of the “text” column

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,:;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [0]:
def clean_text(text):
  text = BeautifulSoup(text, "lxml").text # HTML decoding
  text = text.lower() # lowercase text
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
  text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
  text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
  return text

In [0]:
stemmer = SnowballStemmer("english")

def stemming(sentence):
  stemSentence = ""
  #print(sentence)
  for word in sentence.split():
    stem = stemmer.stem(word)
    stemSentence += stem
    stemSentence += " "
    #stemSentence = stemSentence.strip()
  #print(stemSentence)
  return stemSentence

In [0]:
# Remove Numberic & junk characters
import re

def remove_num(t):
    removed_num_text = re.sub(r'\d+', '', t)
    removed_num_text = re.sub(r"[,@\'?\-\+\.$%_]", '', removed_num_text)
    return removed_num_text

a. Remove unwanted characters  
b. Convert text to lowercase  
c. Remove unwanted spaces

d. Remove stopwords

In [0]:
blog_df['proper_review']=blog_df['text'].apply(clean_text)

In [0]:
blog_df.head(5)

,id,gender,age,topic,sign,date,text,proper_review
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found + 100 pages 45 mb pdf files wait un...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoos toolbar capture urls popupswhich...


In [0]:
blog_df['clean_review'] = blog_df['proper_review'].apply( lambda t : remove_num(t) )
blog_df['clean_review'].head(10)

0    info found   pages  mb pdf files wait untill t...
1    team members drewes van der laag urllink mail ...
2    het kader van kernfusie op aarde maak je eigen...
3                                      testing testing
4    thanks yahoos toolbar capture urls popupswhich...
5    interesting conversation dad morning talking k...
6    somehow cocacola way summing things well early...
7    anything korea country extremes everything see...
8    take read news article urllink joongang ilbo n...
9    surf english news sites lot looking tidbits ko...
Name: clean_review, dtype: object

In [0]:
blog_df['clean_review_stemming']=blog_df['clean_review'].apply(stemming)

In [0]:
blog_df['clean_review_stemming']

0         info found page mb pdf file wait until team le...
1         team member drew van der laag urllink mail rui...
2         het kader van kernfusi op aard maak je eigen w...
3                                                test test 
4         thank yahoo toolbar captur url popupswhich mea...
                                ...                        
681279    dear susan could write realli bitter diatrib d...
681280    dear susan second yeast infect past two month ...
681281            dear susan boyfriend fuck bald good luck 
681282    dear susan clarifi ask leav hous shit piss flo...
681283    hey everybodyand susan might alreadi know name...
Name: clean_review_stemming, Length: 681284, dtype: object

In [0]:
blog_df.drop(['proper_review','text','id','date','clean_review'],axis=1,inplace=True)

In [0]:
blog_df.head(10)

,gender,age,topic,sign,clean_review_stemming
0,male,15,Student,Leo,info found page mb pdf file wait until team le...
1,male,15,Student,Leo,team member drew van der laag urllink mail rui...
2,male,15,Student,Leo,het kader van kernfusi op aard maak je eigen w...
3,male,15,Student,Leo,test test
4,male,33,InvestmentBanking,Aquarius,thank yahoo toolbar captur url popupswhich mea...
5,male,33,InvestmentBanking,Aquarius,interest convers dad morn talk korean put mone...
6,male,33,InvestmentBanking,Aquarius,somehow cocacola way sum thing well earli s fl...
7,male,33,InvestmentBanking,Aquarius,anyth korea countri extrem everyth seem fadbas...
8,male,33,InvestmentBanking,Aquarius,take read news articl urllink joongang ilbo no...
9,male,33,InvestmentBanking,Aquarius,surf english news site lot look tidbit korea f...


In [0]:
# Remove Rare words, which will not contribute to our model
all_words = ' '.join(blog_df['clean_review_stemming'] ).split()
rarely = pd.Series(all_words).value_counts()[-160000:]
rarely.sort_values

<bound method Series.sort_values of visitingm        1
andpaulin        1
uneccisari       1
scriptedfunni    1
guysokthat       1
                ..
ratherb          1
decontruction    1
attackappar      1
greywhit         1
newsvalliseem    1
Length: 160000, dtype: int64>

In [0]:
# remove rare words
# remove frequenct words
blog_df['clean_review_stemming'] = blog_df['clean_review_stemming'].apply( lambda t : ' '.join( word for word in t.split() 
                                                      if word not in rarely) )
blog_df['clean_review_stemming'].head(10)

0    info found page mb pdf file wait until team le...
1    team member drew van der laag urllink mail rui...
2    het kader van kernfusi op aard maak je eigen w...
3                                            test test
4    thank yahoo toolbar captur url popupswhich mea...
5    interest convers dad morn talk korean put mone...
6    somehow cocacola way sum thing well earli s fl...
7    anyth korea countri extrem everyth seem think ...
8    take read news articl urllink joongang ilbo no...
9    surf english news site lot look tidbit korea f...
Name: clean_review_stemming, dtype: object

In [0]:
import textblob
from textblob import Word
from textblob import TextBlob

In [0]:
# Spelling correction  -- 
# !!!!!!!!!!!!!!!!!!!!!!!!  Warning: This step is has taken too long few hours
blog_df['clean_review_stemming'].apply( lambda t :  str( TextBlob(t).correct() ) )

In [0]:
# Lemmatization ; its preferred over stemming because if finds the root word
# perform lemmatization on the title
blog_df['clean_review_stemming']= blog_df['clean_review_stemming'].apply( lambda t : " ".join( [Word(word).lemmatize() for word in t.split()  ]) )
blog_df['clean_review_stemming'].head(5)

In [0]:
blog_df.to_csv(project_path+'/blogtext_cleaned_1.csv')

In [0]:
blog_df.to_csv(project_path+'/blogtext_cleaned_2.csv')

In [0]:
blog_df=pd.read_csv(project_path+'/blogtext_cleaned_1.csv')

In [9]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 6 columns):
Unnamed: 0               681284 non-null int64
gender                   681284 non-null object
age                      681284 non-null int64
topic                    681284 non-null object
sign                     681284 non-null object
clean_review_stemming    677240 non-null object
dtypes: int64(2), object(4)
memory usage: 31.2+ MB


In [10]:
blog_df.shape

(681284, 6)

## Label columns to merge: “gender”, “age”, “topic”, “sign” 

In [0]:
blog_df['label']=blog_df['gender'].map(str)+','+blog_df['age'].map(str)+','+blog_df['topic'].map(str)+','+blog_df['sign'].map(str)

In [12]:
blog_df.head(10)

,Unnamed: 0,gender,age,topic,sign,clean_review_stemming,label
0,0,male,15,Student,Leo,info found page mb pdf file wait until team...,"male,15,Student,Leo"
1,1,male,15,Student,Leo,team member drew van der laag urllink mail rui...,"male,15,Student,Leo"
2,2,male,15,Student,Leo,het kader van kernfusi op aard maak je eigen w...,"male,15,Student,Leo"
3,3,male,15,Student,Leo,test test,"male,15,Student,Leo"
4,4,male,33,InvestmentBanking,Aquarius,thank yahoo toolbar captur url popupswhich mea...,"male,33,InvestmentBanking,Aquarius"
5,5,male,33,InvestmentBanking,Aquarius,interest convers dad morn talk korean put mone...,"male,33,InvestmentBanking,Aquarius"
6,6,male,33,InvestmentBanking,Aquarius,somehow cocacola way sum thing well earli s fl...,"male,33,InvestmentBanking,Aquarius"
7,7,male,33,InvestmentBanking,Aquarius,anyth korea countri extrem everyth seem fadbas...,"male,33,InvestmentBanking,Aquarius"
8,8,male,33,InvestmentBanking,Aquarius,take read news articl urllink joongang ilbo no...,"male,33,InvestmentBanking,Aquarius"
9,9,male,33,InvestmentBanking,Aquarius,surf english news site lot look tidbit korea f...,"male,33,InvestmentBanking,Aquarius"


In [0]:
final_blog=blog_df.drop(['gender', 'age','topic','sign'], axis = 1)

In [0]:
sample_blog=final_blog.sample(frac=1,replace=False,random_state=1)

In [0]:
counts = final_blog['label'].value_counts()

sample_blog = final_blog[~final_blog['label'].isin(counts[counts <400].index)]

In [16]:
sample_blog.shape

(428802, 3)

## Separate features and labels, and split the data into training and testing

In [0]:
X=sample_blog.clean_review_stemming

In [0]:
y=sample_blog.label

In [19]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.10,shuffle=True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(385921,)
(42881,)
(385921,)
(42881,)


In [20]:
X_train

202996    finallyth pictur first round savannah st patri...
577127    strang yes offens probabl sure unbeliev horrib...
456366    cassandra smell like dosent realli shes sit sa...
17708     hand stink cut ass load garlic yesterday gross...
450898    especi sooo right yama ditch kristi happen soo...
                                ...                        
413026    well met great your ever nyc must see even ame...
577500    today grandma th birthday feel deserv fun birt...
214835    today watch return king mood tire music ashita...
236897    procrastin way get move thing ive put logic th...
197282    worm worm worm place open oh that right admit ...
Name: clean_review_stemming, Length: 385921, dtype: object

## Vectorize the features

In [21]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2),max_features=10000)
X_train_dtm = vect.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_train_dtm.shape

(385921, 10000)

In [0]:
#from sklearn.feature_extraction.text import TfidfTransformer
#tfidf_transformer = TfidfTransformer()
#X_tfidf = tfidf_transformer.fit_transform(X_train_dtm)

In [23]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test.apply(lambda x: np.str_(x)))
X_test_dtm.shape

(42881, 10000)

In [0]:
X_tfidf=X_train_dtm
X_test_tfidf=X_test_dtm

In [0]:
#X_test_tfidf = tfidf_transformer.fit_transform(X_test_dtm)

## Transform the labels

In [0]:
y_train = [set(i.split(',')) for i in y_train]

In [0]:
y_test = [set(i.split(',')) for i in y_test]

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [30]:
mlb = MultiLabelBinarizer()
y_train_transform=mlb.fit_transform(y_train)
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '42', '43', '45', '46',
       '48', 'Accounting', 'Advertising', 'Aquarius', 'Aries', 'Arts',
       'Banking', 'Cancer', 'Capricorn', 'Chemicals',
       'Communications-Media', 'Consulting', 'Education', 'Engineering',
       'Fashion', 'Gemini', 'Government', 'Internet', 'Law',
       'LawEnforcement-Security', 'Leo', 'Libra', 'Marketing',
       'Museums-Libraries', 'Non-Profit', 'Pisces', 'Publishing',
       'RealEstate', 'Religion', 'Sagittarius', 'Science', 'Scorpio',
       'Sports-Recreation', 'Student', 'Taurus', 'Technology',
       'Telecommunications', 'Virgo', 'female', 'indUnk', 'male'],
      dtype=object)

In [31]:
y_train_transform

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 1, 0]])

In [0]:
y_test_transform=mlb.transform(y_test)

In [33]:
y_test[1]

{'25', 'Aries', 'indUnk', 'male'}

In [34]:
y_test_transform[1]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [35]:
X_train_dtm.shape

(385921, 10000)

In [36]:
X_tfidf.shape

(385921, 10000)

## Choose a classifier 

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing

In [0]:
#clf=LogisticRegression(solver='lbfgs')
clf=LogisticRegression(C=1.0, penalty='l1', dual=False, solver='liblinear')

In [0]:
clf=OneVsRestClassifier(clf)

In [40]:
clf.fit(X_tfidf, y_train_transform)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [41]:
import joblib
filename = project_path+'/finalized_model_LG.sav'
joblib.dump(clf, filename)

['/content/drive/My Drive/SNLP_R9_Project/finalized_model_LG.sav']

## Fit the classifier, make predictions and get the accuracy 

In [0]:
# make class predictions for X_test_dtm
y_test_predicted = clf.predict(X_test_tfidf)

In [45]:
print("Test Accuracy = ",accuracy_score(y_test_transform,y_test_predicted)*100)

Test Accuracy =  2.3810079055992164


In [47]:
# calculate predicted probabilities for X_test_dtm
y_pred_prob = clf.predict_proba(X_test_tfidf)[:, 1]
y_pred_prob

array([0.12098192, 0.00943096, 0.00461277, ..., 0.00880189, 0.02520089,
       0.02623245])

In [48]:
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print("F1 score Micro= ",f1_score(y_test_transform,y_test_predicted, average="micro"))
print("F1 score Macro= ",f1_score(y_test_transform,y_test_predicted, average="macro"))
print("F1 score Weighted= ",f1_score(y_test_transform,y_test_predicted, average="weighted"))
print("Hamming loss = ",hamming_loss(y_test_transform,y_test_predicted))
print("Precision Score Micro = ",precision_score(y_test_transform,y_test_predicted,average="micro"))
print("Precision Score Macro = ",precision_score(y_test_transform,y_test_predicted,average="macro"))
print("Precision Score Weighted = ",precision_score(y_test_transform,y_test_predicted,average="weighted"))
print("Recall Score Micro = ",recall_score(y_test_transform,y_test_predicted,average="micro"))
print("Recall Score Macro = ",recall_score(y_test_transform,y_test_predicted,average="macro"))
print("Recall Score Weighted = ",recall_score(y_test_transform,y_test_predicted,average="weighted"))

F1 score Micro=  0.4069761093214985
F1 score Macro=  0.2301910203745282
F1 score Weighted=  0.34496169378050745
Hamming loss =  0.056287925647315586
Precision Score Micro =  0.6146252974297359
Precision Score Macro =  0.46096471480430534
Precision Score Weighted =  0.5549306077643296
Recall Score Micro =  0.30420232737109676
Recall Score Macro =  0.17230062015576103
Recall Score Weighted =  0.30420232737109676


## Printing true predict for 5 samples

In [0]:
y_test_pred_inverse=mlb.inverse_transform(y_test_predicted)
y_test_inversed = mlb.inverse_transform(y_test_transform)

In [50]:
y_test_predicted[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [51]:
for i in range(5):
  print(i)
  print(y_test_pred_inverse[i])
  print(y_test_inversed[i])

0
('female', 'indUnk')
('14', 'Student', 'Virgo', 'female')
1
('female', 'indUnk')
('25', 'Aries', 'indUnk', 'male')
2
('24', 'female', 'indUnk')
('24', 'Virgo', 'indUnk', 'male')
3
('indUnk', 'male')
('24', 'Aries', 'indUnk', 'male')
4
('24', 'Technology', 'indUnk', 'male')
('26', 'Leo', 'Technology', 'male')
